# Adapting Preprocessing Script for SageMaker

To build a pipeline, we need to start with individual steps. Our first step will be data preprocessing — the same preprocessing you've done locally in previous courses, but now adapted to run in SageMaker's managed environment.

We'll create a separate file called `data_processing.py` that contains our preprocessing logic. The preprocessing logic itself is identical to what you've done before — we're still capping outliers, creating new features, and splitting the data. The key changes are in how we handle file paths to work within SageMaker's processing environment:

These specific paths (`/opt/ml/processing/`) are SageMaker conventions that allow the service to automatically handle data movement between S3 and your processing containers. When SageMaker runs this script, it will automatically mount your S3 data to the input directory and upload the results from the output directories back to S3.



# Understanding SageMaker Sessions

Before building your pipeline, you need to understand that SageMaker Pipelines require two different types of sessions that serve distinct purposes:



In [ ]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

# Create a SageMaker session for immediate operations
sagemaker_session = sagemaker.Session()

# Create a pipeline session for pipeline components
pipeline_session = PipelineSession()

The distinction between these sessions is about execution context, not timing:

- `sagemaker.Session()` — This is your direct connection to AWS services. When you use this session, you're telling SageMaker "execute this operation using real AWS resources right now." It handles immediate operations like uploading data to S3, creating pipeline definitions, starting executions, and checking status.

- `PipelineSession()` — This is a special "recording" session that creates placeholder operations instead of real ones. When you use this session with processors or estimators, instead of immediately creating SageMaker jobs, it returns step objects that become part of your pipeline definition. These placeholders get converted into real operations only when the pipeline executes.

Without `PipelineSession()`, if you created a processor with a regular session, SageMaker would immediately try to spin up compute instances and start processing your data before you've even finished defining your pipeline! The PipelineSession() lets you define all your pipeline components as a complete workflow first, then execute everything in the proper order when you're ready.

Simple Rule:

- Use `PipelineSession()` for any processor, estimator, or transformer that should become a pipeline step
- Use sagemaker.Session() for immediate actions like managing the pipeline itself

In [ ]:
# Get the default SageMaker bucket name
default_bucket = sagemaker_session.default_bucket()

# Local file path
DATA_PATH = "data/california_housing.csv"

# S3 prefix (folder path within the bucket)
DATA_PREFIX = "datasets"

try:
    # Upload the dataset using the upload_data() method
    s3_uri = sagemaker_session.upload_data(
        path=DATA_PATH,
        bucket=default_bucket,
        key_prefix=DATA_PREFIX
    )
    
    print(f"Data uploaded successfully to: {s3_uri}")
    
except Exception as e:
    print(f"Error: {e}")

# Setting Up AWS Resources

Now that we understand sessions, we need to set up the AWS resources and permissions that our pipeline will use:



In [ ]:
# Retrieve your AWS account ID (used for constructing resource ARNs)
account_id = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']

# Get the default S3 bucket for your SageMaker resources
default_bucket = sagemaker_session.default_bucket()

# Define the SageMaker execution role ARN
SAGEMAKER_ROLE = f"arn:aws:iam::{account_id}:role/SageMakerDefaultExecution"

# Creating the Processing Environment

To run our preprocessing script in SageMaker, we need to define the computing environment where our code will execute. We use an `SKLearnProcessor` because our preprocessing script uses scikit-learn libraries, and crucially, we use the `PipelineSession` to ensure the processor becomes a pipeline step rather than executing immediately:



In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

# Create a processor that will run our data preprocessing script
processor = SKLearnProcessor(
    framework_version="1.2-1",    # Specify scikit-learn version
    role=SAGEMAKER_ROLE,          # IAM role with necessary permissions
    instance_type="ml.m5.large",  # Compute instance type for processing
    instance_count=1,             # Number of instances to use
    sagemaker_session=pipeline_session  # Use pipeline session for deferred execution
)

# Building Our First Pipeline Step

With our processor configured, we can now create the actual processing step using `ProcessingStep`. This step defines what data goes in, what comes out, and what code runs in between.

Before proceeding, note that we assume you have already uploaded your raw dataset (`california_housing.csv`) to your S3 default bucket at the path `/datasets/california_housing.csv`. This is necessary because the pipeline will read the input data directly from S3.

In [ ]:
from sagemaker.workflow.steps import ProcessingStep

# Define the processing step with inputs, outputs, and the script to run
processing_step = ProcessingStep(
    name="ProcessData",   # Unique name for this step in the pipeline
    processor=processor,  # The processor we defined above
    inputs=[
        # Define where the raw data comes from (S3 location)
        sagemaker.processing.ProcessingInput(
            source=f"s3://{default_bucket}/datasets/california_housing.csv",
            destination="/opt/ml/processing/input"  # Where data will be mounted in container
        )
    ],
    outputs=[
        # Define where processed training data will be saved
        sagemaker.processing.ProcessingOutput(
            output_name="train_data",               # Reference name for this output
            source="/opt/ml/processing/train"       # Container path where script saves data
        ),
        # Define where processed test data will be saved
        sagemaker.processing.ProcessingOutput(
            output_name="test_data",                # Reference name for this output
            source="/opt/ml/processing/test"        # Container path where script saves data
        )
    ],
    code="data_processing.py"  # The Python script that performs the processing
)

Notice how the paths in our `ProcessingInput` and `ProcessingOutput` definitions perfectly match the paths our script expects. The `inputs` parameter specifies where our raw data comes from (an S3 location) and where it will be mounted inside the processing container (`/opt/ml/processing/input`). The `outputs` parameter defines where our processed data will be saved, with separate outputs for training and test data. Each output has a name that we can reference later and a source path where our processing script will write the data. The `code` parameter points to the Python script that contains our preprocessing logic.

This connection between the step definition and the script paths is crucial — it's what allows SageMaker to automatically handle all the data movement for you.

# Creating the Pipeline

With our processing step defined, we can now create our first pipeline. A pipeline is simply a collection of steps that execute in order, and right now we have just one step. Note that we use the regular `sagemaker_session` for pipeline management:



In [ ]:
from sagemaker.workflow.pipeline import Pipeline

# Set a name for the SageMaker Pipeline
PIPELINE_NAME = "california-housing-preprocessing-pipeline"

# Create pipeline with our processing step
pipeline = Pipeline(
    name=PIPELINE_NAME,
    steps=[processing_step],  # For now, just one step
    sagemaker_session=sagemaker_session  # Use regular session for pipeline management
)

This creates a pipeline definition with our single processing step. In future lessons, we'll add more steps to this list to create more complex workflows with training, evaluation, and model registration.

At this point, we've only created the pipeline definition in memory — it doesn't exist in AWS yet. To make it available in SageMaker, we need to register it with the service using the `upsert` method:

In [ ]:
# Create or update pipeline (upsert = update if exists, create if not)
pipeline.upsert(role_arn=SAGEMAKER_ROLE)

The `upsert` method is particularly useful because it handles both creation and updates intelligently. If this is the first time you're running this code, it will create a new pipeline in SageMaker. If you run the same code again after making changes to your pipeline definition, it will update the existing pipeline rather than throwing an error. This makes iterative development much smoother — you can modify your pipeline code, run it again, and SageMaker will automatically apply your changes.

Think of the pipeline definition as a blueprint or recipe. Once you've registered this blueprint with SageMaker using `upsert`, you can execute it multiple times. Each execution is a separate run of the same blueprint, potentially with different data or parameters.

# Executing the Pipeline

Now that our pipeline is registered with SageMaker, we can start an execution. This is where the actual work begins:

In [ ]:
# Start pipeline execution and get execution object for monitoring
execution = pipeline.start()

# Get the execution ARN for tracking
print(f"Pipeline execution ARN: {execution.arn}")

When you call `pipeline.start()`, SageMaker immediately begins executing your pipeline in the background. This means your local Python script doesn't need to wait for the processing to complete — the heavy computational work is happening on AWS infrastructure while your script continues running or even after it finishes.

The execution object provides valuable information about the running pipeline. You'll see output similar to:

`
Pipeline execution ARN: arn:aws:sagemaker:us-east-1:123456789012:pipeline/california-housing-preprocessing-pipeline/execution/x1gc33lgj8v5
`

# Monitoring Your Pipeline

In [ ]:
# Check the current status
execution_details = execution.describe()

# Display status
print(f"Status: {execution_details['PipelineExecutionStatus']}")